In [1]:
import numpy as np
from matplotlib import pyplot
import math
import pandas as pd

In [2]:
L=3         #Length of Nozzle
N=31        #No. of nodes
dx=L/(N-1)
C=0.5       #Courant Number
rho=np.zeros(N)         #density
V=np.zeros(N)           #velocity
T=np.zeros(N)           #temperature
Time=1400               #Timesteps
a=np.zeros(N)           #Non dimensional speed of sound
A=np.zeros(N)           #Cross-section area
M=np.zeros(N)           #Mach Number
P=np.zeros(N)           #Pressure
gamma=1.4               #cp/cv
point=np.zeros(N)
distratio=np.zeros(N)
t=0                     #currenttimestep

In [3]:
#Nozzle shape
for i in range(N):
    A[i]=1+2.2*(i*dx-1.5)*(i*dx-1.5)
    point[i]=i
    distratio[i]=i*dx/L

In [4]:
#Initial Conditions
for i in range(N):
    rho[i]=1-0.3146*i*dx
    T[i]=1-0.2314*i*dx
    V[i]=(0.1+1.09*i*dx)*pow(T[i],0.5)

In [5]:
dt=np.zeros(N)
a=np.sqrt(T)
#gradients
drho1=np.zeros(N)
drho2=np.zeros(N)
dV1=np.zeros(N)
dV2=np.zeros(N)
dT1=np.zeros(N)
dT2=np.zeros(N)
lnA=np.zeros(N)
#initializing predicted values
prho=np.zeros(N)
pV=np.zeros(N)
pT=np.zeros(N)

In [6]:
#loop
for t in range (Time):
    if t%100==00:
         print(t)
   
    for i in range (N):
        dt[i]=C*dx/(a[i]+V[i])
    DT=min(dt)
    #predictor
    lnA=np.log(A)
    for i in range (0,N-1):
        drho1[i]=-rho[i]*(V[i+1]-V[i])/dx-rho[i]*V[i]*(lnA[i+1]-lnA[i])/dx-V[i]*(rho[i+1]-rho[i])/dx
        dV1[i]=-V[i]*(V[i+1]-V[i])/dx-1/gamma*((T[i+1]-T[i])/dx+T[i]/rho[i]*(rho[i+1]-rho[i])/dx)
        dT1[i]=-V[i]*(T[i+1]-T[i])/dx-(gamma-1)*T[i]*((V[i+1]-V[i])/dx+V[i]*(lnA[i+1]-lnA[i])/dx)
    for i in range (0,N-1):
        prho[i]=rho[i]+drho1[i]*DT
        pV[i]=V[i]+dV1[i]*DT
        pT[i]=T[i]+dT1[i]*DT
    for i in range (1,N):
        #corrector
        drho2[i]=-prho[i]*(pV[i]-pV[i-1])/dx-prho[i]*pV[i]*(lnA[i]-lnA[i-1])/dx-pV[i]*(prho[i]-prho[i-1])/dx
        dV2[i]=-pV[i]*(pV[i]-pV[i-1])/dx-1/gamma*((pT[i]-pT[i-1])/dx+pT[i]/prho[i]*(prho[i]-prho[i-1])/dx)
        dT2[i]=-pV[i]*(pT[i]-pT[i-1])/dx-(gamma-1)*pT[i]*((pV[i]-pV[i-1])/dx+pV[i]*(lnA[i]-lnA[i-1])/dx)
    for i in range (1,N):
        rho[i]=rho[i]+DT*0.5*(drho1[i]+drho2[i])
        V[i]=V[i]+DT*0.5*(dV1[i]+dV2[i])
        T[i]=T[i]+DT*0.5*(dT1[i]+dT2[i])
    
    #boundary conditions
    V[0]=2*V[1]-V[2]
    rho[0]=1
    T[0]=1
    V[N-1]=2*V[N-2]-V[N-3]
    rho[N-1]=2*rho[N-2]-rho[N-3]
    T[N-1]=2*T[N-2]-T[N-3]
    a=np.sqrt(T)
    M=V/a
    P=rho*T
    

0
100
200


C:\Users\Tarun\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars


300
400
500
600
700
800
900
1000
1100
1200
1300


In [7]:
print("Final Result")
Result={'I':point,'x/L':distratio,'A/ThroatArea':A,'rho/rho0':rho,'V/V0':V,'T/T0':T,'P/P0':P,'M':M}
Resulttable=pd.DataFrame(Result)
Resulttable

Final Result


,I,x/L,A/ThroatArea,rho/rho0,V/V0,T/T0,P/P0,M
0,0.0,0.000000,5.950,1.000000,0.098607,1.000000,1.000000,0.098607
1,1.0,0.033333,5.312,0.997527,0.111934,0.999006,0.996535,0.111990
2,2.0,0.066667,4.718,0.996662,0.125261,0.998663,0.995330,0.125345
3,3.0,0.100000,4.168,0.993972,0.142572,0.997582,0.991569,0.142745
4,4.0,0.133333,3.662,0.991304,0.162270,0.996513,0.987847,0.162553
5,5.0,0.166667,3.200,0.987025,0.186529,0.994789,0.981882,0.187017
6,6.0,0.200000,2.782,0.981484,0.215435,0.992555,0.974176,0.216241
7,7.0,0.233333,2.408,0.973469,0.250746,0.989307,0.963059,0.252097
8,8.0,0.266667,2.078,0.962302,0.293555,0.984758,0.947635,0.295818
9,9.0,0.300000,1.792,0.946357,0.345756,0.978210,0.925736,0.349585
